In [24]:
url = "https://www.rightmove.co.uk/property-to-rent/find.html?locationIdentifier=REGION%5E93727&maxBedrooms=0&minBedrooms=0&maxPrice=1200&propertyTypes=&includeLetAgreed=false&mustHave=&dontShow=&furnishTypes=&keywords="

In [25]:
headers = {
    'User-Agent': 'curl/7.64.1',  # Example User-Agent header from curl
    # Add other headers observed from the curl command
}

response = requests.get(url, headers=headers)

In [26]:
len(response.content)

496578

In [31]:
with open("sample.xml", "w") as f:
    f.write(response.content.decode("utf-8"))

In [16]:
import requests

# API endpoint
url = "https://www.rightmove.co.uk/property-to-rent/find.html"
# Parameters
params = {
    "searchType": "RENT",
    "locationIdentifier": "REGION^87490",
    "insId": "1",
    "radius": "0.0",
    "minPrice": "",
    "maxPrice": "",
    "minBedrooms": "",
    "maxBedrooms": "",
    "displayPropertyType": "",
    "maxDaysSinceAdded": "",
    "sortByPriceDescending": "",
    "_includeLetAgreed": "on",
    "primaryDisplayPropertyType": "",
    "secondaryDisplayPropertyType": "",
    "oldDisplayPropertyType": "",
    "oldPrimaryDisplayPropertyType": "",
    "letType": "",
    "letFurnishType": "",
    "houseFlatShare": ""
}

headers = {
    'User-Agent': 'curl/7.64.1',  # Example User-Agent header from curl
    # Add other headers observed from the curl command
}

# Sending GET request
response = requests.get(url, params=params, headers=headers)

# Checking if the request was successful
if response.status_code == 200:
    print("Request successful!")
    # You can process the response here, for example, print the content
    # print(response.text)
else:
    print(f"Request failed with status code: {response.status_code}")


Request successful!


In [119]:
import xml.etree.ElementTree as ET

import bs4
from bs4 import BeautifulSoup

In [37]:
soup = BeautifulSoup(content, 'html.parser')

In [69]:
def get_tags(root):
    children = [c for c in root.children if isinstance(c, bs4.Tag)]
    return children

children = get_tags(soup)

In [93]:
def get_cleaned_soup(path):
    with open(path) as f:
        content = f.read()

    soup = BeautifulSoup(content, 'html.parser')
    
    for script in soup(["script", "style", "noscript"]):
        script.decompose()

    for svg in soup.find_all('svg'):
        svg.decompose()

    for select_tag in soup.find_all("select"):
        select_tag.decompose()

    for div in soup.find_all('div'):
        if div.find('a', {'class': 'seo-nav-subNav-link'}):
            div.decompose()
        if div.find('a', {'class': 'seo-mob-nav-user'}):
            div.decompose()
        if div.find('a', {'class': 'seo-mob-nav-sign-in'}):
            div.decompose()

    for div_group in soup.find_all('div', class_='seo-mob-nav-link-group'):
        div_group.decompose()

    for div_group in soup.find_all('div', class_='seo-nav-item overseas'):
        div_group.decompose()

    filters_tray_div = soup.find('div', class_='filtersTray')
    if filters_tray_div:
        filters_tray_div.decompose()

    filters_tray_div = soup.find('div', class_='searchLayoutControls-label')
    if filters_tray_div:
        filters_tray_div.decompose()

    filters_tray_div = soup.find('div', class_='searchLayoutControls-container')
    if filters_tray_div:
        filters_tray_div.decompose()

    return soup

cleaned_soup = get_cleaned_soup("sample.xml")

In [95]:
f = open("cleaned_sample.xml", "w")
f.write(cleaned_soup.prettify())
f.close()

In [107]:
def get_search_results(soup):
    listings = soup.find_all('div', class_='l-searchResult')
    l_info = []
    
    for listing in listings:
        # Initialize a dictionary to store the extracted information
        listing_info = {}
    
        # Extract the property ID
        listing_info['property_id'] = listing.get('id', '').split('-')[-1]
    
        # Extract the image URLs
        images = listing.find_all('img')
        listing_info['image_urls'] = [img['src'] for img in images if 'src' in img.attrs]
    
        # Extract the description
        description = listing.find('span', {'itemprop': 'description'})
        listing_info['description'] = description.text.strip() if description else ''
    
        # Extract the price
        price = listing.find('span', class_='propertyCard-priceValue')
        listing_info['price'] = price.text.strip() if price else ''
    
        # Extract the added date
        added_date = listing.find('span', class_='propertyCard-branchSummary-addedOrReduced')
        listing_info['added_date'] = added_date.text.strip() if added_date else ''
    
        # Extract the contact phone number
        phone_number = listing.find('a', class_='propertyCard-contactsPhoneNumber')
        listing_info['phone_number'] = phone_number.text.strip() if phone_number else ''
    
        # Print or store the extracted information
        l_info.append(listing_info)
    return l_info
    
cards = get_search_results(cleaned_soup)

In [118]:
print(cleaned_soup.find('div', class_='l-searchResult').prettify())

<div class="l-searchResult is-list" data-bind="attr: { 'id': 'property-' + id() }, css: { 'is-hidden': hasNoDetails, 'is-list': isList}" data-test="propertyCard-0" id="property-144595010">
 <div class="propertyCard" data-bind="css: {'propertyCard--premium': premiumListing, 'propertyCard-hideMobileIconsMVT': hideMobileIcons, 'propertyCard--featured': featuredProperty, 'propertyCard--saved': isPropertySaved }" itemscope="" itemtype="http://schema.org/Residence">
  <a class="propertyCard-anchor" id="prop144595010">
  </a>
  <div class="propertyCard-wrapper">
   <div class="propertyCard-images">
    <div class="propertyCard-main-img-mask aspect-3x2">
     <div class="propertyCard-main-img">
      <a class="propertyCard-img-link aspect-3x2" data-test="property-img" href="/properties/144595010#/?channel=RES_LET">
       <div class="propertyCard-img">
        <span class="no-svg-camera camera">
        </span>
        <img alt="Property Image 1" itemprop="image" src="https://media.rightmove.c

In [113]:
cards[0]

{'property_id': '144595010',
 'image_urls': ['https://media.rightmove.co.uk:443/dir/crop/10:9-16:9/260k/259202/144595010/259202_THECI_005196_IMG_00_0000_max_476x317.jpeg',
  ''],
 'description': 'A fully furnished double bedroom is available for couples with balcony in a four-bedroom with one bathroom house-share. This property is conveniently situated in zone 2, providing excellent transportation connections to central London, with easy access to Upper Holloway station nearby.',
 'price': '£1,127 pcm',
 'added_date': 'Added on 10/02/2024',
 'phone_number': '020 4538 5940'}

In [101]:
def get_listings(soup):
    property_cards = soup.find_all('div', class_='propertyCard')
    return property_cards
    
cards = get_listings(cleaned_soup)